![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **Text Analysis of Beer Reviews**
Here we use pySpark to analyze the text in the commercial description and review text to create similarity scores between beers.  The scores can then be used for clustering and beer style identification or to find beers similar to what a user enjoys, as a recommendation service.

### **Preliminaries**
#### We read in the allBeer.txt file and create an RDD consisting of lines.
#### We want to remove the header from the file, so the parseDataFileLine function identifies lines starting with 'beer_id' and applies a flag of 0, other lines with the correct number of fields are flagged 1, and incorrect lines are flagged -1.  The lines are split into arrays.

In [1]:
import re

def parseDatafileLine(datafileLine):
    ##Parse a line of the data file using the specified regular expression pattern
    splitArray = datafileLine.split("\t")
    for x in range(0,len(splitArray)):
        splitArray[x]=splitArray[x].replace("\"",'')
    #print len(splitArray)
    #print splitArray[0],splitArray[1],splitArray[2]
    if splitArray[0]=='beer_id':
        return (splitArray,0)
    elif len(splitArray)<>23:
        ##this is a failed parse
        return (splitArray,-1)
    else:
        return (splitArray, 1)

### Reading the file
#### We read the file into three rdds by first parsing the file as above, the header rdd, failed rdd and the valid rdd.  Print the header names so we can remember what fields we're dealing with and in what order.

In [2]:
import sys
import os

baseDir = os.path.join('')
allBeer_Path = 'AllBeer.txt'
STOPWORDS_PATH = 'stopwords.txt'

def parseData(filename):
    #Parse a data file returns a RDD of parsed lines
    
    return (sc
            .textFile(filename, 4, 0)
            .map(parseDatafileLine)
            .cache())

def loadData(path):
    ##Load a data file, returns a RDD of parsed valid lines
    
    filename = os.path.join(baseDir, path)
    raw = parseData(filename).cache()
    failed = (raw
              .filter(lambda s: s[1] == -1)
              .map(lambda s: s[0]))
    for line in failed.take(10):
        print '%s - Invalid datafile line: %s' % (path, line)
    valid = (raw
             .filter(lambda s: s[1] == 1)
             .map(lambda s: s[0])
             .cache())
    header = (raw
              .filter(lambda s: s[1]==0)
             .map(lambda s:s[0])
             )
    for line in header.take(1):
        for x in range(0,len(line)):
            print x,line[x]
            
    rawLines = raw.count()
    validLines = valid.count()
    failedLines = failed.count()
    print '%s - Read %d lines, successfully parsed %d lines, failed to parse %d lines' % (path, rawLines, validLines,failedLines)
    return valid
    
allBeer = loadData(allBeer_Path)
#allReviews = loadData(allReviews_Path)

0 beer_id
1 beer_name
2 brewer_name
3 beer_style
4 distribution
5 brewery_location
6 commercial_desc
7 RATINGS: 
8 MEAN (/5)
9 WEIGHTED AVG
10 EST. CALORIES
11 ABV (%)
12 IBU
13 SCORE
14 AROMA (/10)
15 APPEARANCE(/5)
16 TASTE(/10)
17 PALATE(/5)
18 OVERALL(/20)
19 reviewer_name
20 review_location
21 review_date
22 review_content
AllBeer.txt - Read 240355 lines, successfully parsed 240354 lines, failed to parse 0 lines


### Let's examine the first few entries of a sample of 5 lines to check if things look ok.

In [3]:
sampleArray=allBeer.takeSample(False,5,1)
for line in sampleArray:
    print len(line)
    print 'allBeer: %s, %s, %s, %s, %s\n' % (line[0], line[1], line[2],line[3],line[4])

23
allBeer: 40920, Wernecker Haustrunk Pils, Wernecker Bierbrauerei, Pilsener, distribution unknown

23
allBeer: 21911, Au Ma'tre Brasseur La Boucaneuse, AMB - Ma'tre Brasseur, Smoked, distribution unknown

23
allBeer: 4991, New Albanian / Struise Naughty Girl, New Albanian Brewing Company, India Pale Ale (IPA), Regional Distribution

23
allBeer: 38430, BrewDog IPA is Dead - Pioneer, BrewDog, India Pale Ale (IPA), Broad Distribution

23
allBeer: 41353, Cascade Cerise Nouveau, Cascade Brewing, Sour/Wild Ale, Local Distribution



### Now we'll split the data into a training set (80%) and test set (%20).  
#### This is slightly complicated by the fact that we want to split each user into 80/20, not the set of reviews as a whole.  We will take advantage of stratified sampling in Spark, grouping the reviews by the user name, then sampling by key.  To get the unused data we employ subtractByKey using a compound key of the username and the beer_id, guaranting uniqueness.

In [4]:
##Using the allBeer array, take stratified sample, and remove blank reviews.
beerByUser = allBeer.map(lambda x:(x[19],x)).filter(lambda (x,y):y[22]!='')
sampleKeys = beerByUser.keys().collect()
fractions={}
for k in sampleKeys:
    fractions[k]=0.8
    
beerTrain = beerByUser.sampleByKey(False,fractions).cache()
beerTrainKeyed = beerTrain.map(lambda (x,y):(y[0]+y[19],y))
beerTest = allBeer.map(lambda x:(x[0]+x[19],x)).subtractByKey(beerTrainKeyed).map(lambda (x,y):(y[19],y)).cache()
print beerByUser.count()
print beerTrain.count()
print beerTest.count()

240334
192623
47731


### Let's examine the results with a couple of random users.

In [5]:
from numpy.random import random_integers
##find a random user and print out the train and test set.
randomUsers = random_integers(1,len(sampleKeys),2)
sampleUserReviewCount = beerByUser.filter(lambda (x,y):x==sampleKeys[randomUsers[0]]).count()
sampleUserTrainCount = beerTrain.filter(lambda (x,y):x==sampleKeys[randomUsers[0]]).count()
sampleUserTestCount = beerTest.filter(lambda (x,y):x==sampleKeys[randomUsers[0]]).count()
print "The user %s has %d reviews, split into %d Train and %d Test" % (sampleKeys[randomUsers[0]],sampleUserReviewCount,sampleUserTrainCount,sampleUserTestCount)

sampleUserReviewCount = beerByUser.filter(lambda (x,y):x==sampleKeys[randomUsers[1]]).count()
sampleUserTrainCount = beerTrain.filter(lambda (x,y):x==sampleKeys[randomUsers[1]]).count()
sampleUserTestCount = beerTest.filter(lambda (x,y):x==sampleKeys[randomUsers[1]]).count()
print "The user %s has %d reviews, split into %d Train and %d Test" % (sampleKeys[randomUsers[1]],sampleUserReviewCount,sampleUserTrainCount,sampleUserTestCount)

The user mzaar has 72 reviews, split into 60 Train and 12 Test
The user Bosbouw has 86 reviews, split into 71 Train and 15 Test


### Evaluate the fit
#### So 0.1839 spearman correlation, this isn't so good, but it's a positive correlation which shows that there is some correlation between the order of beers we generate and the order of beers the user actually prefers.  What could be the issue, why is this number so low?  We have not yet added non textual features, such as the alcohol content, the style, the bitterness and the brewery, which could all play a factor in what user's like.  We also have a lot of extraneous words which contribute to the cosine distance, but have no real meaning in determining a user's preference for a beer, words such as 'yesterday'.  We also have many words that only appear once or twice in the set of reviews such as another user's name "bob4532 gave me this beer to review" which are also meaningless.

In [7]:
##First need to make integer usernames since ALS doesn't support text userIDs.
##Solution: create a hash dictionary
#print beerTrain.take(5)
userList = allBeer.map(lambda x: x[19]).collect()
userList = list(set(userList))
userDict={}
for x in range(0,len(userList)):
    userDict[userList[x]]=x
print userDict.items()[0:5]
print len(userDict)

[('nazzty', 0), ('Bewitched', 1), ('Nejhleader', 8686), ('CObiased', 3), ('DeKreeft2799', 8687)]
10420


In [38]:
#0 beer_id
#1 beer_name
#2 brewer_name
#3 beer_style
#4 distribution
#5 brewery_location
#6 commercial_desc
#7 RATINGS: 
#8 MEAN (/5)
#9 WEIGHTED AVG
#10 EST. CALORIES
#11 ABV (%)
#12 IBU
#13 SCORE
#14 AROMA (/10)
#15 APPEARANCE(/5)
#16 TASTE(/10)
#17 PALATE(/5)
#18 OVERALL(/20)
#19 reviewer_name
#20 review_location
#21 review_date
#22 review_content

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

# Reshape data
ratings = beerTrain.map(lambda l: Rating(int(l[1][0]), int(userDict[l[0]]), float(l[1][13])))
#print ratings.takeSample(False,5,1)
# Build the recommendation model using Alternating Least Squares
rank = 80
numIterations = 40
model = ALS.train(ratings, rank, numIterations)

# Evaluate the model on training data - bad form!
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Training Mean Squared Error = " + str(MSE))

# Save and load model
#model.save(sc, "myModelPath")
#sameModel = MatrixFactorizationModel.load(sc, "myModelPath")


##Evaluate the model on the test data
testRatings = beerTest.map(lambda l: Rating(int(l[1][0]), int(userDict[l[0]]), float(l[1][13])))
#print testRatings.takeSample(5,5,False)
testdata = testRatings.map(lambda p: (p[0],p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = testRatings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
#print ratesAndPreds.takeSample(False,5,1)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Test Mean Squared Error = " + str(MSE))

Training Mean Squared Error = 0.000420930268195
Test Mean Squared Error = 0.281912698633


In [39]:
predsByUser = ratesAndPreds.map(lambda r: (r[0][1],(r[0][0],r[1][0],r[1][1])))

In [40]:
### ToDo: lookup Kendall Tau, Rank Biased Overlap (RBO), maybe mean squared error??
from scipy.stats import spearmanr
from scipy.stats.stats import rankdata
from scipy.stats import pearsonr
import math

### Need to implement a function which will take the results from the cosSim RDD and spit out
### average?? spearman.

def catTuplesToLists(tupleOne,tupleTwo):
    if len(tupleOne)==len(tupleTwo):
        a=[]
        for x in range(0,len(tupleOne)):
            firstEle=tupleOne[x]
            if type(firstEle) is not list: firstEle = [ tupleOne[x] ]
            secondEle=tupleTwo[x]
            if type(secondEle) is not list: secondEle = [ tupleTwo[x] ]
            a.append(firstEle+secondEle)
        return tuple(a)
    else:
        raise ValueError('Two tuples are not the same dimensions')

def mattSpearman(listOne,listTwo):
    if type(listOne) is list and type(listTwo) is list:
        lO=len(listOne)
        if lO==len(listTwo) and lO>1:
            lOR = rankdata(listOne)
            lTR = rankdata(listTwo)
            print lOR
            print lTR
            dSquared=0
            for x in range(0,lO):
                dSquared+=(lOR[x]-lTR[x])**2
            return 1-6*dSquared/lO/(lO**2-1)
        else:
            raise ValueError("Both Lists must be same length and greater than one.")
    else:
        return 0

def reduceNaN(a,b):
    if math.isnan(a):
        a=0
    if math.isnan(b):
        b=0
    return a+b

def avgSpearman(inputRDD):
    #So the problem here is trying to output two lists which have the scores in the same order.
    #Write custom reduce function, catTuplesToLists
    convertedToLists = inputRDD.reduceByKey(catTuplesToLists)
    #print convertedToLists.take(5)
    filteredLists = convertedToLists.filter(lambda (x,y):type(y[1]) is list)
    #print filteredLists.take(5)
    spearmanByKey = filteredLists.map(lambda (x,y):(x,mattSpearman(y[1],y[2]), len(y[1])))
    #print spearmanByKey.take(5)
    spearmanOnly = spearmanByKey.map(lambda (x,y,z):y)
    #print spearmanOnly.count()
    #print spearmanOnly.take(5)
    avgSpearmanRho=spearmanOnly.reduce(lambda a,b:a+b)/spearmanOnly.count()
    return avgSpearmanRho

print "The Spearman correlation using collaborative filtering using only scores is %f" % avgSpearman(predsByUser)

The Spearman correlation using collaborative filtering using only scores is 0.442987
